In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
driver = './chromedriver.exe'

In [ ]:
browser = webdriver.Chrome(driver)

In [ ]:
## DATA COLLECTION
titles = []
locations = []
companies = []
salaries = []
all_li_text = []
jobtitles = []
i = 0

urls = ['https://www.indeed.co.in/jobs?q=machine+learning&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=artificial+intelligence&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=data+science&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=data+analytics&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=web+development&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=django+developer&l=India&start={i}',
        'https://www.indeed.co.in/jobs?q=react+js&l=India&start={i}']

In [ ]:
for j in urls:  
    
    titles = []
    locations = []
    companies = []
    salaries = []
    all_li_text = []
    jobtitles = []

    for i in range(0, 500, 10):
        sleep(2.5)
        browser.get(j.format(i=i))
        cards = browser.find_elements_by_class_name('result')
        for info in cards:
            details = BeautifulSoup(info.get_attribute('innerHTML'), 'html.parser')

            try:
                title = details.find('a', class_ = 'jobtitle').text.replace('\n', '')
                titles.append(title)
            except:
                titles.append(None)

            try:
                location = details.find(class_ = 'location').text.replace('\n', '')
                locations.append(location)
            except:
                locations.append(None)

            try:
                company = details.find(class_ = 'company').text.replace('\n', '')
                companies.append(company)
            except:
                companies.append(None)

            try:
                salary = details.find(class_ = 'salary').text.replace('\n', '')
                salaries.append(salary)
            except:
                salaries.append(None)

            info.click()
            sleep(1)

            try:
                des = browser.find_element_by_id("vjs-desc")
                decHTML = BeautifulSoup(des.get_attribute('innerHTML'), 'html.parser')
                all_li = decHTML.find_all('li')

                for i in all_li:
                    all_li_text.append(i.text.replace('\n',''))
                jobtitles.append(all_li_text)

            except Exception as e:
                
                jobtitles.append(None)

    dic = {
    'Titles':titles,
    'Locations':locations,
    'Companies':companies,
    'Salaries':salaries,
    'JobDescription':jobtitles
    }
    
    df = pd.DataFrame(dic)
    df.to_csv(re.findall(r'[a-z]*.\+[a-z]*', j)[0])

In [ ]:
## Handling popups

In [ ]:
for j in urls:  
    
    titles = []
    locations = []
    companies = []
    salaries = []
    all_li_text = []
    jobtitles = []

    for i in range(0, 500, 10):
        browser.get(j.format(i=i))
        sleep(2.5)
        
        while True:
            try:
                cards = browser.find_elements_by_class_name('result')
                break
            except:              
                popUp = browser.find_element_by_id('popover-x')
                popUp.click()
                   
        for info in cards:
            details = BeautifulSoup(info.get_attribute('innerHTML'), 'html.parser')

            try:
                title = details.find('a', class_ = 'jobtitle').text.replace('\n', '')
                titles.append(title)
            except:
                titles.append(None)

            try:
                location = details.find(class_ = 'location').text.replace('\n', '')
                locations.append(location)
            except:
                locations.append(None)

            try:
                company = details.find(class_ = 'company').text.replace('\n', '')
                companies.append(company)
            except:
                companies.append(None)

            try:
                salary = details.find(class_ = 'salary').text.replace('\n', '')
                salaries.append(salary)
            except:
                salaries.append(None)

           while True:
            try:
                
                info.click()
                sleep(1)

                try:
                    des = browser.find_element_by_id("vjs-desc")
                    decHTML = BeautifulSoup(des.get_attribute('innerHTML'), 'html.parser')
                    all_li = decHTML.find_all('li')

                    for i in all_li:
                    all_li_text.append(i.text.replace('\n',''))
                    jobtitles.append(all_li_text)

                except Exception as e:
                    jobtitles.append(None)
                break
                
            except:
                popUp = browser.find_element_by_id('popover-x')
                popUp.click()


    dic = {
    'Titles':titles,
    'Locations':locations,
    'Companies':companies,
    'Salaries':salaries,
    'JobDescription':jobtitles
    }
    
    df = pd.DataFrame(dic)
    df.to_csv(re.findall(r'[a-z]*.\+[a-z]*', j)[0])

In [ ]:
df1 = pd.read_csv('ai.csv')                   # File names are different and will be depending on the context
df2 = pd.read_csv('data_analyst.csv')
df3 = pd.read_csv('datascience2.csv')
df4 = pd.read_csv ('machinelearning.csv')

In [ ]:
df = pd.concat([df1, df2, df3, df4], axis = 0)
df = df.drop_duplicates()

In [ ]:
grp = df.groupby('Company').count()['Title'].sort_values(ascending = False)[:10][::-1]   # Finding the first 10 comapnies with max job offers

In [ ]:
grp.plot(kind = 'barh', figsize = (15,5)) # Visualising the same

In [ ]:
#####################Cleaning the Data using NLP #################

In [ ]:
stop_words = stopwords.words('english')

def Clean(desc):
    words = word_tokenize(str(desc))
    imp_words = []
    for word in words:
        lower = word.lower()
        if lower not in stop_words:
            imp_words.append(lower)
    return imp_words

In [ ]:
tags = df['Description'].dropna().apply(Clean)
res = tags.apply(Counter)
f_res = res.sum().items()
f_res = list(f_res)
f_res = sorted(f_res, key = lambda kv:kv[1], reverse = True)    

In [ ]:
series = pd.Series({
    k:v for k,v in f_res
})

skills = ['python', 'opencv', 'pandas', 'math', 'nltk']
Filter = series.filter(items = skills)[::-1]         # skills list contain set of skills mentioned in the job description

In [ ]:
Filter.plot(kind = 'barh', figsize = (10,5))           # Visualizing the filtered data